___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4


#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange


1. Підключитися до API.
2. Отримати курс долара за останній рік.

In [1]:
import requests
from datetime import timedelta, datetime

def get_curs4period(ndays, endday, currency='USD'):
    """
    Returns db = <dic> {'YYYYmmdd': <dic> {'data': datatime, 'rate': Exchange rate on the date of the currency 'currency'}
    for 'ndays' and last day 'endday'
    """
    
    db = {}
    url = 'https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange'

    for day in range(ndays):
        dt = endday - timedelta(days=day)
        strdt = dt.strftime('%Y%m%d')
        par = {
            'valcode': currency,
            'date': strdt,
            'json': 'json'
        }
        r = requests.get(url, params = par)
        if r.status_code == 200:
            res = r.json()
            db[strdt] = {
                'date': dt,
                'rate': res[0]['rate']
            }
        else: db[strdt] = None

    return db

Today = datetime.today()
USDdb = get_curs4period(365, Today)

3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [2]:
import statistics as st

def stat_month(db, month):
    """
    Returns curs for month
    """
    temp = [v['rate'] for k,v in db.items() if v['date'].month == month]
    mean = st.mean(temp)
    std  = st.stdev(temp)
    return mean, std

MonthStat = []

for i in range(1,13):
    m, s = stat_month(USDdb,i)
    MonthStat.append({i: [m, s]})
    print(f"Місяць - {i}: середній курс - {round(m,2)} грн/USD, стандартне відхилення - {round(s,2)} грн/USD")

Місяць - 1: середній курс - 27.98 грн/USD, стандартне відхилення - 0.59 грн/USD
Місяць - 2: середній курс - 28.41 грн/USD, стандартне відхилення - 0.46 грн/USD
Місяць - 3: середній курс - 29.25 грн/USD, стандартне відхилення - 0.0 грн/USD
Місяць - 4: середній курс - 29.25 грн/USD, стандартне відхилення - 0.0 грн/USD
Місяць - 5: середній курс - 29.25 грн/USD, стандартне відхилення - 0.0 грн/USD
Місяць - 6: середній курс - 29.25 грн/USD, стандартне відхилення - 0.0 грн/USD
Місяць - 7: середній курс - 27.58 грн/USD, стандартне відхилення - 0.84 грн/USD
Місяць - 8: середній курс - 26.78 грн/USD, стандартне відхилення - 0.11 грн/USD
Місяць - 9: середній курс - 26.73 грн/USD, стандартне відхилення - 0.15 грн/USD
Місяць - 10: середній курс - 26.37 грн/USD, стандартне відхилення - 0.11 грн/USD
Місяць - 11: середній курс - 26.45 грн/USD, стандартне відхилення - 0.36 грн/USD
Місяць - 12: середній курс - 27.21 грн/USD, стандартне відхилення - 0.1 грн/USD


4. Дану інформацію записати у файл за допомогою pickle.

In [3]:
import pickle

with open('db.pkl', 'wb') as file:
    pickle.dump(USDdb, file)

### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [4]:
import requests
from bs4 import BeautifulSoup

URL = 'https://smallpacking.agrosem.ua/shop/'
HEADERS = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36'
}


def get_html(url, params=''):
    """Return soup-obj with html from url-request"""
    r = requests.get(url, headers=HEADERS, params=params)
    if r.status_code == 200:
        return r
    else:
        return None
    

def get_quantity(soup):
    """Return quantity of product"""
    vaga_attrbut = soup.find('tr', class_='woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_vaga')
    if None == vaga_attrbut:
        quantity = None
        units = None
    else:
        valst = vaga_attrbut.td.text.split()
        quantity = float(valst[0].replace(',','.'))
        units = valst[1].lower()
    return quantity, units


def get_products(html):
    """Return database with products from page"""
    soup = BeautifulSoup(html.text, 'lxml')
    items = soup.find_all('div', class_='product-from-category-container')
    products = []

    for item in items:
        href = item.a.get('href')
        item_number = href.split('/')[-2]

        try:
            price = int(item.find('span', class_='regular-price').text.split(' грн')[0].replace(' ', ''))
        except ValueError:
            price = None

        prod_html = get_html(href)
        if prod_html != None:
            prod_soup = BeautifulSoup(prod_html.text, 'lxml')
            quantity, units = get_quantity(prod_soup)
        else:
            print(f"Quantity of product '{item_number}' wasn't read")
            quantity, units = None, None

        products.append(
            {
                'item':   item_number,
                'name':   item.a.span.text,
                'price':  price,
                'weight': quantity,
                'units':  units
            }
        )
        
    return products


def parcer():
    """Return databese of all products"""
    db = []
    page = 1
    html = get_html(URL)
    
    while html != None: # url and product pages is available
        print(f"  Parcing page {page}")
        db.extend(get_products(html))  
        print(f"    current products quantity is - {len(db)}")
        page += 1
        html = get_html(URL+f'page/{page}/')

    return db

db = parcer()
print(f"Products quantity is - {len(db)}")
    

  Parcing page 1
    current products quantity is - 12
  Parcing page 2
    current products quantity is - 24
  Parcing page 3
    current products quantity is - 36
  Parcing page 4
    current products quantity is - 48
  Parcing page 5
    current products quantity is - 60
  Parcing page 6
    current products quantity is - 72
  Parcing page 7
    current products quantity is - 84
  Parcing page 8
    current products quantity is - 96
  Parcing page 9
    current products quantity is - 108
  Parcing page 10
    current products quantity is - 120
  Parcing page 11
    current products quantity is - 132
  Parcing page 12
    current products quantity is - 144
  Parcing page 13
    current products quantity is - 156
  Parcing page 14
    current products quantity is - 168
  Parcing page 15
    current products quantity is - 180
  Parcing page 16
    current products quantity is - 192
  Parcing page 17
    current products quantity is - 204
  Parcing page 18
    current products quantity 

### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [5]:
import sqlite3
import pandas as pd

db_file = 'tysql.sqlite'

with sqlite3.connect('tysql.sqlite') as conn:
    curs = conn.cursor()
    print(curs.description)

def query(sql_query):
    """ Запит до бази данних """
    with sqlite3.connect(db_file) as conn:
        curs = conn.cursor()
        curs.execute(sql_query)
        response = curs.fetchall()
    return response

def get_columns_name(table_name):
    sql_query = f"PRAGMA table_info({table_name})"
    res = query(sql_query)
    column_name_list = [x[1] for x in res]
    return column_name_list

None


2. Виведіть перелік всіх таблиць.

In [6]:
sql_q = """SELECT * FROM sqlite_master
           WHERE type = 'table';"""

tables_info = query(sql_q)
tables_list = [tb[1] for tb in tables_info[0:-1]]

print('List of Tables in the database:')
for tab in tables_list:
    print('  ',tab)

List of Tables in the database:
   Customers
   OrderItems
   Orders
   Products
   Vendors


3. Список всіх cust_id з таблиці Customers table.

In [7]:
sql_q = """SELECT cust_id 
           FROM Customers;"""
response = query(sql_q)
pd.DataFrame(response, columns=['cust_id'])

,cust_id
0,1000000001
1,1000000002
2,1000000003
3,1000000004
4,1000000005


4. Всю таблицю Customers table

In [8]:
sql_q = """SELECT * FROM Customers;"""
response = query(sql_q)
pd.DataFrame(response, columns=get_columns_name("Customers"))

,cust_id,cust_name,cust_address,cust_city,cust_state,cust_zip,cust_country,cust_contact,cust_email
0,1000000001,Village Toys,200 Maple Lane,Detroit,MI,44444,USA,John Smith,sales@villagetoys.com
1,1000000002,Kids Place,333 South Lake Drive,Columbus,OH,43333,USA,Michelle Green,None
2,1000000003,Fun4All,1 Sunny Place,Muncie,IN,42222,USA,Jim Jones,jjones@fun4all.com
3,1000000004,Fun4All,829 Riverside Drive,Phoenix,AZ,88888,USA,Denise L. Stephens,dstephens@fun4all.com
4,1000000005,The Toy Store,4545 53rd Street,Chicago,IL,54545,USA,Kim Howard,None


5. Список клієнтів (cust_names) відсортованих від Z до A

In [9]:
sql_q = """SELECT cust_name 
           FROM Customers
           ORDER BY cust_name;"""
response = query(sql_q)
pd.DataFrame(response, columns=['cust_name'])

,cust_name
0,Fun4All
1,Fun4All
2,Kids Place
3,The Toy Store
4,Village Toys


6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [10]:
sql_q = """SELECT cust_id, order_num, order_date 
           FROM Customers INNER JOIN Orders USING(cust_id)
           ORDER BY cust_id, order_date;"""
response = query(sql_q)
pd.DataFrame(response, columns=['cust_id', 'order_num', 'order_date'])

,cust_id,order_num,order_date
0,1000000001,20009,2012-02-08
1,1000000001,20005,2012-05-01
2,1000000003,20006,2012-01-12
3,1000000004,20007,2012-01-30
4,1000000005,20008,2012-02-03


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [11]:
sql_q = """SELECT order_num, order_item, prod_name, quantity,  item_price
           FROM OrderItems INNER JOIN Products USING(prod_id)
           ORDER BY quantity DESC, item_price DESC;"""
response = query(sql_q)
pd.DataFrame(response, columns=['order_num', 'order_item', 'prod_name', 'quantity',  'item_price'])

,order_num,order_item,prod_name,quantity,item_price
0,20009,1,Fish bean bag toy,250,2.49
1,20009,2,Bird bean bag toy,250,2.49
2,20009,3,Rabbit bean bag toy,250,2.49
3,20005,2,18 inch teddy bear,100,10.99
4,20005,1,8 inch teddy bear,100,5.49
5,20007,2,Fish bean bag toy,100,2.99
6,20007,3,Bird bean bag toy,100,2.99
7,20007,4,Rabbit bean bag toy,100,2.99
8,20007,1,18 inch teddy bear,50,11.49
9,20007,5,Raggedy Ann,50,4.49


8. Товар (з таблиці Products), ціна якого становить 9.49

In [12]:
sql_q = """SELECT prod_name, prod_price
           FROM Products
           WHERE prod_price = 9.49;"""
response = query(sql_q)
pd.DataFrame(response, columns=['prod_name', 'prod_price'])

,prod_name,prod_price
0,King doll,9.49
1,Queen doll,9.49


9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [13]:
sql_q = """SELECT prod_name, prod_price
           FROM Products
           WHERE prod_price BETWEEN 3 AND 6
           ORDER BY prod_price;"""
response = query(sql_q)
pd.DataFrame(response, columns=['prod_name', 'prod_price'])

,prod_name,prod_price
0,Fish bean bag toy,3.49
1,Bird bean bag toy,3.49
2,Rabbit bean bag toy,3.49
3,Raggedy Ann,4.99
4,8 inch teddy bear,5.99


10. Кількість товару, що було продано

In [14]:
sql_q = """SELECT SUM(quantity)
           FROM OrderItems;"""
response = query(sql_q)
print(f'Number of products sold - {response[0][0]} pcs.')

Number of products sold - 1430 pcs.


11. Кількість найменувань товару, ціна якого більша за 4

In [15]:
sql_q = """SELECT count(*)
           FROM Products
           WHERE prod_price > 4;"""
response = query(sql_q)
print(f'Number of products with a price higher than 4$ - {response[0][0]} pcs.')

Number of products with a price higher than 4$ - 6 pcs.


12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [16]:
sql_q = """SELECT prod_name, prod_price
           FROM Products
           ORDER BY prod_price DESC
           LIMIT 3;"""
response = query(sql_q)
pd.DataFrame(response, columns=['prod_name', 'prod_price'])

,prod_name,prod_price
0,18 inch teddy bear,11.99
1,King doll,9.49
2,Queen doll,9.49


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [17]:
sql_q = """SELECT cust_id, cust_name, count(*)
           FROM Customers INNER JOIN Orders USING(cust_id)
           GROUP BY cust_id
           ORDER BY cust_name;"""
response = query(sql_q)
pd.DataFrame(response, columns=['cust_id', 'cust_name', 'quantity_of_orders'])

,cust_id,cust_name,quantity_of_orders
0,1000000003,Fun4All,1
1,1000000004,Fun4All,1
2,1000000005,The Toy Store,1
3,1000000001,Village Toys,2


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [18]:
sql_q = """SELECT cust_id, cust_name, order_num
           FROM Customers INNER JOIN Orders USING(cust_id)
           ORDER BY cust_id;"""
response = query(sql_q)
pd.DataFrame(response, columns=['cust_id', 'cust_name', 'order_num'])

,cust_id,cust_name,order_num
0,1000000001,Village Toys,20005
1,1000000001,Village Toys,20009
2,1000000003,Fun4All,20006
3,1000000004,Fun4All,20007
4,1000000005,The Toy Store,20008


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [20]:
sql_q = """SELECT prod_name, prod_price
           FROM Products
           WHERE prod_price < 5
           UNION
           SELECT prod_name, prod_price
           FROM Products
           WHERE prod_price >= 5;"""
response = query(sql_q)
pd.DataFrame(response, columns=['prod_name', 'prod_price'])

,prod_name,prod_price
0,12 inch teddy bear,8.99
1,18 inch teddy bear,11.99
2,8 inch teddy bear,5.99
3,Bird bean bag toy,3.49
4,Fish bean bag toy,3.49
5,King doll,9.49
6,Queen doll,9.49
7,Rabbit bean bag toy,3.49
8,Raggedy Ann,4.99


### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди

**Завдання 1**

*Найдите номер модели, скорость и размер жесткого диска для всех ПК стоимостью менее 500 дол. Вывести: model, speed и hd*  
`SELECT model, speed, hd`  
`FROM PC`  
`WHERE price < 500;`  
 
**Завдання 2**

*Найдите производителей принтеров. Вывести: maker*   
`SELECT DISTINCT maker`  
`FROM Product`  
`WHERE type='Printer';`  

**Завдання 3**

*Найдите номер модели, объем памяти и размеры экранов ПК-блокнотов, цена которых превышает 1000 дол.*  
`SELECT model, ram, screen`  
`FROM Laptop`  
`WHERE price > 1000;` 

**Завдання 4**

*Найдите все записи таблицы Printer для цветных принтеров.*  
`SELECT *`  
`FROM Printer`  
`WHERE color = 'y';`  

**Завдання 5**

*Найдите номер модели, скорость и размер жесткого диска ПК, имеющих 12x или 24x CD и цену менее 600 дол.*  
`SELECT model, speed, hd`  
`FROM PC`  
`WHERE (cd IN ('12x', '24x')) and (price < 600);`  

**Завдання 6**

*Для каждого производителя, выпускающего ПК-блокноты c объёмом жесткого диска не менее 10 Гбайт, найти скорости таких ПК-блокнотов. Вывод: производитель, скорость.*  
`SELECT DISTINCT maker, speed`  
`FROM Product INNER JOIN Laptop ON Product.model=Laptop.model`  
`WHERE hd >= 10;`

**Завдання 7**

*Найдите номера моделей и цены всех имеющихся в продаже продуктов (любого типа) производителя B (латинская буква).*  
`SELECT DISTINCT Product.model, price`  
`FROM Product`  
`   INNER JOIN PC ON Product.model = PC.model`  
`WHERE maker = 'B'`  
`UNION`  
`SELECT DISTINCT Product.model, price`  
`FROM Product`  
`   INNER JOIN Laptop ON Product.model = Laptop.model`  
`WHERE maker = 'B'`  
`UNION`  
`SELECT DISTINCT Product.model, price`  
`FROM Product`  
`   INNER JOIN Printer ON Product.model = Printer.model`  
`WHERE maker = 'B';`  

**Завдання 8**

*Найдите производителя, выпускающего ПК, но не ПК-блокноты.*  
`SELECT maker`  
`FROM Product`  
`WHERE type = 'PC'`  
`EXCEPT`  
`SELECT maker`  
`FROM Product`  
`WHERE type = 'Laptop';`  

**Завдання 9**

*Найдите производителей ПК с процессором не менее 450 Мгц. Вывести: Maker*  
`SELECT DISTINCT maker`  
`FROM Product`  
`     INNER JOIN PC ON Product.model = PC.model`  
`WHERE speed >= 450;`

**Завдання 10**

*Найдите модели принтеров, имеющих самую высокую цену. Вывести: model, price*  
`SELECT Printer.model, price`  
`FROM Product`  
`     INNER JOIN Printer ON Product.model = Printer.model`  
`WHERE price = (`  
`     SELECT MAX(price)`  
`     FROM Printer`  
`)`  

**Завдання 11**

*Найдите среднюю скорость ПК.*  
`SELECT AVG(speed)`  
`FROM PC`  

**Завдання 12**

*Найдите среднюю скорость ПК-блокнотов, цена которых превышает 1000 дол.*  
`SELECT AVG(speed)`  
`FROM Laptop`  
`WHERE price > 1000;`  

**Завдання 13**

*Найдите среднюю скорость ПК, выпущенных производителем A.*  
`SELECT AVG(speed)`  
`FROM Product`  
`     INNER JOIN PC ON Product.model = PC.model`  
`WHERE maker = 'A';`  

**Завдання 14**

*Найдите класс, имя и страну для кораблей из таблицы Ships, имеющих не менее 10 орудий.*  
`SELECT Ships.class, name, country`  
`FROM Classes`  
`     INNER JOIN Ships ON Classes.class = Ships.class`  
`WHERE numGuns >= 10;`  

**Завдання 15**

*Найдите размеры жестких дисков, совпадающих у двух и более PC. Вывести: HD*  
`SELECT hd`  
`FROM PC`  
`GROUP BY hd`  
`HAVING count(*) > 1;`  

**Завдання 16**

*Найдите пары моделей PC, имеющих одинаковые скорость и RAM. В результате каждая пара указывается только один раз, т.е. (i,j), но не (j,i), Порядок вывода: модель с большим номером, модель с меньшим номером, скорость и RAM.*  
`SELECT DISTINCT A.model, B.model, A.speed, A.ram`  
`FROM PC AS A, PC AS B`  
`WHERE (A.model > B.model)`  
`   AND (A.speed = B.speed)`
`   AND (A.ram = B.ram);`  

**Завдання 17**

*Найдите модели ПК-блокнотов, скорость которых меньше скорости каждого из ПК. Вывести: type, model, speed*  
`SELECT DISTINCT type, Laptop.model, speed`  
`FROM Product`  
`     INNER JOIN Laptop ON Product.model = Laptop.model`  
`WHERE speed < ALL(SELECT speed FROM PC);`  

**Завдання 18**

*Найдите производителей самых дешевых цветных принтеров. Вывести: maker, price*  
`SELECT DISTINCT maker, price`  
`FROM Product`  
`     INNER JOIN Printer ON Product.model = Printer.model`  
`WHERE color = 'y'`  
`  AND price = (SELECT MIN(price)`  
`               FROM Printer`  
`               WHERE color = 'y');`  

**Завдання 19**

*Для каждого производителя, имеющего модели в таблице Laptop, найдите средний размер экрана выпускаемых им ПК-блокнотов. Вывести: maker, средний размер экрана.*  
`SELECT maker, AVG(screen)`  
`FROM Product`  
`     INNER JOIN Laptop ON Product.model = Laptop.model`  
`GROUP BY maker;`  

**Завдання 20**

*Найдите производителей, выпускающих по меньшей мере три различных модели ПК. Вывести: Maker, число моделей ПК.*  
`SELECT maker, count(PC.model)`  
`FROM Product`  
`     INNER JOIN PC ON Product.model = PC.model`  
`GROUP BY maker`  
`HAVING count(PC.model) >= 3;`  

На мій погляд алгоритм вірний, але сайт його не прийняв. В той же час правильний результат запиту який вказує сайт не відповідає базі данних. Або щось я не розумію тут.

**Завдання 21**

*Найдите максимальную цену ПК, выпускаемых каждым производителем, у которого есть модели в таблице PC. Вывести: maker, максимальная цена.*  
`SELECT maker, MAX(PC.price)`  
`FROM Product`  
`     INNER JOIN PC ON Product.model = PC.model`  
`GROUP BY maker;`  

**Завдання 22**

*Для каждого значения скорости ПК, превышающего 600 МГц, определите среднюю цену ПК с такой же скоростью. Вывести: speed, средняя цена.*  
`SELECT speed, AVG(price)`  
`FROM PC`  
`GROUP BY speed`  
`HAVING speed > 600;`  

**Завдання 23**

*Найдите производителей, которые производили бы как ПК со скоростью не менее 750 МГц, так и ПК-блокноты со скоростью не менее 750 МГц. Вывести: Maker*  
`SELECT DISTINCT maker`  
`FROM Product`  
`     INNER JOIN PC ON Product.model = PC.model`  
`WHERE speed >= 750`  
`INTERSECT`  
`SELECT DISTINCT maker`  
`FROM Product`  
`     INNER JOIN Laptop ON Product.model = Laptop.model`  
`WHERE speed >= 750;`  

**Завдання 24**

*Перечислите номера моделей любых типов, имеющих самую высокую цену по всей имеющейся в базе данных продукции.*  
`WITH Un(model, price) AS (`  
`  SELECT model, price`  
`  FROM PC`  
`  UNION ALL`  
`  SELECT model, price`  
`  FROM Laptop`  
`  UNION ALL`  
`  SELECT model, price`  
`  FROM Printer)`  
`SELECT DISTINCT model`  
`FROM Un`  
`WHERE price = (SELECT MAX(price) FROM Un);`  

**Завдання 25**

*Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker*  
`WITH Un(maker, speed) AS (`  
`     SELECT maker, speed`  
`     FROM Product INNER JOIN PC ON Product.model = PC.model`  
`     WHERE ram = (SELECT MIN(ram) FROM PC)`  
`)`  
`SELECT DISTINCT maker`  
`FROM Product`  
`WHERE type = 'Printer'`  
`  AND maker IN (SELECT maker`  
`                FROM Un`  
`                WHERE speed = (SELECT MAX(speed) FROM Un)`  
`)`  

**Завдання 26**

*Найдите среднюю цену ПК и ПК-блокнотов, выпущенных производителем A (латинская буква). Вывести: одна общая средняя цена.*  
`SELECT AVG(price)`  
`FROM(`  
`   SELECT price, maker`  
`   FROM Product INNER JOIN PC ON Product.model = PC.model`  
`   WHERE maker = 'A'`  
`  UNION ALL`  
`   SELECT price, maker`  
`   FROM Product INNER JOIN Laptop ON Product.model = Laptop.model`  
`   WHERE maker = 'A'`  
`) AS Un`  

**Завдання 27**

*Найдите среднюю цену ПК и ПК-блокнотов, выпущенных производителем A (латинская буква). Вывести: одна общая средняя цена.*  
`SELECT maker, AVG(hd)`  
`FROM Product INNER JOIN PC ON Product.model = PC.model`  
`GROUP BY maker`  
`HAVING maker IN (SELECT DISTINCT maker`  
`                 FROM Product`  
`                 WHERE type = 'Printer');`  

**Завдання 28**

*Используя таблицу Product, определить количество производителей, выпускающих по одной модели.*  
`SELECT count(maker) AS qty`  
`FROM(SELECT maker`  
`     FROM(SELECT maker, model`  
`          FROM Product`  
`          GROUP BY maker, model`  
`     ) AS Un1`  
`     GROUP BY maker`
`     HAVING count(maker) = 1`  
`) AS Un2;`  

**Завдання 29**

*В предположении, что приход и расход денег на каждом пункте приема фиксируется не чаще одного раза в день [т.е. первичный ключ (пункт, дата)], написать запрос с выходными данными (пункт, дата, приход, расход). Использовать таблицы Income_o и Outcome_o.*  
`SELECT`  
`CASE`    
`    WHEN Income_o.point  IS NULL THEN Outcome_o.point`  
`    WHEN Outcome_o.point IS NULL THEN Income_o.point`  
`    ELSE Income_o.point`  
`END AS point,`  
`CASE`  
`    WHEN Income_o.date  IS NULL THEN Outcome_o.date`  
`    WHEN Outcome_o.date IS NULL THEN Income_o.date`  
`    ELSE Income_o.date`  
`END AS date,`  
`inc, out`  
`FROM Income_o`  
`     FULL OUTER JOIN Outcome_o`  
`     ON Income_o.date = Outcome_o.date`  
`     AND Income_o.point = Outcome_o.point;`  

**Завдання 30**

*В предположении, что приход и расход денег на каждом пункте приема фиксируется произвольное число раз (первичным ключом в таблицах является столбец code), требуется получить таблицу, в которой каждому пункту за каждую дату выполнения операций будет соответствовать одна строка. Вывод: point, date, суммарный расход пункта за день (out), суммарный приход пункта за день (inc). Отсутствующие значения считать неопределенными (NULL).*  
`SELECT`  
`CASE`  
`    WHEN Inco.point IS NULL THEN Outc.point`  
`    WHEN Outc.point IS NULL THEN Inco.point`  
`    ELSE Inco.point`  
`END AS point,`  
`CASE`  
`    WHEN Inco.date IS NULL THEN Outc.date`  
`    WHEN Outc.date IS NULL THEN Inco.date`  
`    ELSE Inco.date`  
`END AS date,`  
`out AS Outcome, inc AS income`  
`FROM (SELECT point, date, SUM(inc) AS inc`  
`      FROM Income`  
`      GROUP BY point, date`  
`      ) AS Inco`  
`    FULL OUTER JOIN`  
`     (SELECT point, date, SUM(out) AS out`  
`      FROM Outcome`  
`      GROUP BY point, date`  
`      ) AS Outc`  
`     ON  Inco.date = Outc.date`  
`     AND Inco.point = Outc.point`  